## Описание датасета

В работе используется объединённый датасет по публичным российским компаниям, который сочетает рыночные данные (цены акций, капитализация, биржевые мультипликаторы), фундаментальные показатели из T-Invest API и бухгалтерскую финансовую отчётность за 2024 год из базы СПАРК.

### Получение списка торгуемых акций (MOEX) через T-Invest API

В этом шаге мы выгружаем базовый список акций, доступных для торговли, отфильтровывая рублевые обыкновенные акции (без привилегированных), и сохраняем ключевые идентификаторы компаний в CSV.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

SANDBOX_TOKEN = os.getenv("SANDBOX_TOKEN")

In [ ]:
import logging
import csv
from t_tech.invest import Client, InstrumentStatus

TOKEN = SANDBOX_TOKEN
logging.basicConfig(level=logging.INFO)

OUT_FILE = "shares_rub_common.csv"

rows = []

with Client(TOKEN) as client:
    resp = client.instruments.shares(
        instrument_status=InstrumentStatus.INSTRUMENT_STATUS_BASE,
        instrument_exchange=InstrumentStatus.INSTRUMENT_STATUS_UNSPECIFIED,
    )

    for s in resp.instruments:
        if s.currency == "rub" and "привил" not in s.name.lower():
            rows.append({
                "figi": s.figi,
                "ticker": s.ticker,
                "name": s.name,
                "issue_size": s.issue_size,
                "uid": s.uid,
                "assetUid": s.asset_uid,
            })

with open(OUT_FILE, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["figi", "ticker", "name", "issue_size", "currency", "uid", "assetUid"],
    )
    writer.writeheader()
    writer.writerows(rows)

print(f"Saved {len(rows)} rows to {OUT_FILE}")


Saved 151 rows to shares_rub_common.csv


### Загрузка дневных котировок по всем акциям и сохранение в отдельные файлы

На этом этапе для каждой акции из списка загружаются дневные свечи за последний год через T-Invest API, после чего данные сохраняются в отдельные CSV-файлы (по одной компании).


In [ ]:
import csv
import logging
import os
import re
from datetime import timedelta
from pathlib import Path

from t_tech.invest import CandleInterval, Client
from t_tech.invest.caching.market_data_cache.cache import MarketDataCache
from t_tech.invest.caching.market_data_cache.cache_settings import MarketDataCacheSettings
from t_tech.invest.utils import now

TOKEN = SANDBOX_TOKEN  
logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.INFO)

INPUT_CSV = "shares_rub_common.csv"  
OUT_DIR = Path("stocks")
CACHE_DIR = Path("market_data_cache")

DAYS = 365
INTERVAL = CandleInterval.CANDLE_INTERVAL_DAY


def safe_filename(s):
    s = s.strip()
    s = re.sub(r'[<>:"/\\|?*\x00-\x1F]', "_", s) 
    s = re.sub(r"\s+", " ", s)
    return s[:150] if len(s) > 150 else s


def q_to_float(q):
    if q is None:
        return None
    try:
        return float(q.units) + float(q.nano) / 1_000_000_000
    except Exception:
        return None


def read_figis_from_csv(path):
    items = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            figi = (row.get("figi") or "").strip()
            ticker = (row.get("ticker") or "").strip()
            name = (row.get("name") or "").strip()
            if figi:
                items.append((figi, ticker, name))
    return items


def save_candles_for_figi(market_data_cache, figi, out_path):
    candles = list(
        market_data_cache.get_all_candles(
            figi=figi,
            from_=now() - timedelta(days=DAYS),
            interval=INTERVAL,
        )
    )

    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "w", encoding="utf-8-sig", newline="") as f:
        w = csv.writer(f)
        w.writerow(["time", "open", "high", "low", "close", "volume", "is_complete"])
        for c in candles:
            w.writerow([
                c.time.isoformat() if c.time else None,
                q_to_float(getattr(c, "open", None)),
                q_to_float(getattr(c, "high", None)),
                q_to_float(getattr(c, "low", None)),
                q_to_float(getattr(c, "close", None)),
                getattr(c, "volume", None),
                getattr(c, "is_complete", None),
            ])

    return len(candles)


In [ ]:
OUT_DIR.mkdir(parents=True, exist_ok=True)

items = read_figis_from_csv(INPUT_CSV)

with Client(TOKEN) as client:
    settings = MarketDataCacheSettings(base_cache_dir=CACHE_DIR)
    market_data_cache = MarketDataCache(settings=settings, services=client)

    ok = 0
    for figi, ticker, name in items:
        label = ticker or name or figi
        fname = safe_filename(f"{ticker or 'TICKER'}_{figi}.csv")
        out_path = OUT_DIR / fname

        try:
            n = save_candles_for_figi(market_data_cache, figi, out_path)
            logging.info("Saved %s candles for %s (%s) -> %s", n, label, figi, out_path)
            ok += 1
        except Exception as e:
            logging.exception("Failed for %s (%s): %s", label, figi, e)

logging.info("Done. Successfully saved: %d / %d", ok, len(items))

### Выгрузка фундаментальных показателей для всех компаний

На этом шаге мы батчами запрашиваем фундаментальные показатели (мультипликаторы и финансовые метрики) для всех компаний из списка, используя assetUid, и сохраняем результаты в единый CSV-файл.


In [ ]:
import csv
import logging
from datetime import date, datetime
from pathlib import Path

from t_tech.invest import Client, GetAssetFundamentalsRequest

TOKEN = SANDBOX_TOKEN
logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.INFO)

INPUT_CSV = "shares_rub_common.csv"
OUT_CSV = "asset_fundamentals_all.csv"

BATCH_SIZE = 50 


def iso(v):
    if isinstance(v, (datetime, date)):
        return v.isoformat()
    return v


def read_asset_uids(path):
    uids = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        r = csv.DictReader(f)

        possible_cols = ["assetUid", "asset_uid", "uid"]
        cols = r.fieldnames or []
        col = next((c for c in possible_cols if c in cols), None)
        if not col:
            pass

        for row in r:
            uid = (row.get(col) or "").strip()
            if uid:
                uids.append(uid)

    seen = set()
    out = []
    for u in uids:
        if u not in seen:
            seen.add(u)
            out.append(u)
    return out


def obj_to_row(obj):
    d = {}
    for k, v in vars(obj).items():
        d[k] = iso(v)
    return d


def chunked(xs, n):
    for i in range(0, len(xs), n):
        yield xs[i:i + n]


asset_uids = read_asset_uids(INPUT_CSV)
logging.info("Loaded asset UIDs: %d", len(asset_uids))

all_rows = []
with Client(TOKEN) as client:
    for batch in chunked(asset_uids, BATCH_SIZE):
        resp = client.instruments.get_asset_fundamentals(GetAssetFundamentalsRequest(assets=batch))

        for f in resp.fundamentals:
            all_rows.append(obj_to_row(f))

        logging.info("Fetched fundamentals: +%d (total %d)", len(resp.fundamentals), len(all_rows))

fieldnames = sorted({k for row in all_rows for k in row.keys()})

with open(OUT_CSV, "w", encoding="utf-8-sig", newline="") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(all_rows)

logging.info("Saved %d rows to %s", len(all_rows), OUT_CSV)

INFO:root:Loaded asset UIDs: 151
INFO:t_tech.invest.logging:e8482108f2c1f7d7fbfef8ca5c5dd937 GetAssetFundamentals
INFO:root:Fetched fundamentals: +50 (total 50)
INFO:t_tech.invest.logging:d14d21532828cbd6e4d4f5c2cb7e4a98 GetAssetFundamentals
INFO:root:Fetched fundamentals: +49 (total 99)
INFO:t_tech.invest.logging:60df7b6e4089661ab30420da426aa328 GetAssetFundamentals
INFO:root:Fetched fundamentals: +48 (total 147)
INFO:t_tech.invest.logging:aac1095ceca17d2fc8df87cd805ab143 GetAssetFundamentals
INFO:root:Fetched fundamentals: +1 (total 148)
INFO:root:Saved 148 rows to asset_fundamentals_all.csv


### Формирование сводной таблицы с рыночными и фундаментальными показателями

На этом этапе мы объединяем список акций и фундаментальные данные в одну сводную таблицу по asset_uid, упорядочивая ключевые показатели и сохраняя результат в CSV.


In [ ]:
import pandas as pd

SHARES_FILE = "shares_rub_common.csv"
FUND_FILE = "asset_fundamentals_all.csv"
OUT_FILE = "stocks_summary.csv"


shares = pd.read_csv(SHARES_FILE, encoding="utf-8-sig")
funds = pd.read_csv(FUND_FILE, encoding="utf-8-sig")

shares = shares.rename(columns={"assetUid": "asset_uid"})

df = shares.merge(
    funds,
    how="left",
    on="asset_uid",
)

preferred_cols = [
    "figi",
    "ticker",
    "name",
    "asset_uid",
    "issue_size",
    "shares_outstanding",
    "free_float",
    "market_capitalization",
    "pe_ratio_ttm",
    "price_to_book_ttm",
    "price_to_sales_ttm",
    "roe",
    "roa",
    "net_margin_mrq",
    "revenue_ttm",
    "net_income_ttm",
    "total_debt_mrq",
    "total_debt_to_equity_mrq",
    "average_daily_volume_last_10_days",
    "average_daily_volume_last_4_weeks",
    "high_price_last_52_weeks",
    "low_price_last_52_weeks",
    "currency",
]

first_cols = [c for c in preferred_cols if c in df.columns]

rest_cols = [c for c in df.columns if c not in first_cols]

df = df[first_cols + rest_cols]

df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

print("Saved summary table:", OUT_FILE)
print("Rows:", len(df))
print("Columns:", len(df.columns))

missing = [c for c in preferred_cols if c not in df.columns]
if missing:
    print("Missing columns (ok):", missing)

Saved summary table: stocks_summary.csv
Rows: 151
Columns: 62
Missing columns (ok): ['currency']


### Объединение рыночных данных с бухгалтерской отчетностью СПАРК

На этом шаге сводная таблица с рыночными и фундаментальными показателями объединяется с финансовой отчетностью из СПАРКа по биржевому тикеру, при этом сохраняются только компании, присутствующие в обоих источниках, и устраняются дубликаты.

In [ ]:
import pandas as pd

STOCKS_FILE = "stocks_summary.csv"
SPARK_FILE = "СПАРК_Выборка_компаний_20260204_1918.xlsx"
OUT_FILE = "stocks_summary_with_spark.csv"


stocks = pd.read_csv(STOCKS_FILE, encoding="utf-8-sig")
spark = pd.read_excel(SPARK_FILE)

spark = spark.rename(columns={"Тикер биржевой": "ticker"})

stocks["ticker"] = stocks["ticker"].astype(str).str.strip().str.upper()
spark["ticker"] = spark["ticker"].astype(str).str.strip().str.upper()

dup = spark[spark["ticker"].duplicated(keep=False)].copy()
if len(dup) > 0:
    print("SPARK duplicate tickers (count):")
    print(dup["ticker"].value_counts().head(30).to_string())
    print()

sort_cols = []
if "2024, Активы  всего, RUB" in spark.columns:
    sort_cols.append("2024, Активы  всего, RUB")
elif "2024, Выручка, RUB" in spark.columns:
    sort_cols.append("2024, Выручка, RUB")

if sort_cols:
    spark_sorted = spark.sort_values(sort_cols, ascending=False)
    spark_uniq = spark_sorted.drop_duplicates(subset=["ticker"], keep="first")
else:
    spark_uniq = spark.drop_duplicates(subset=["ticker"], keep="first")

stocks_uniq = stocks.drop_duplicates(subset=["ticker"], keep="first")

df = stocks_uniq.merge(
    spark_uniq,
    how="inner",
    on="ticker",
    suffixes=("_api", "_spark"),
    validate="one_to_one" 
)

df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

print("Saved merged table:", OUT_FILE)
print("Rows:", len(df))
print("Columns:", len(df.columns))


SPARK duplicate tickers (count):
ticker
NAN    3779

Saved merged table: stocks_summary_with_spark.csv
Rows: 91
Columns: 154


### Витрина данных: ключевые метрики + последняя цена закрытия

На этом этапе мы добавляем к сводной таблице последнюю доступную цену закрытия из сохраненных файлов со свечами и формируем компактную “витрину” с основными мультипликаторами и ключевыми показателями из СПАРКа.

In [13]:
import pandas as pd
from pathlib import Path

INPUT = Path("stocks_summary_with_spark.csv")
STOCKS_DIR = Path("stocks")

df = pd.read_csv(INPUT, encoding="utf-8-sig")

last_rows = []
for p in STOCKS_DIR.glob("*.csv"):
    ticker = p.stem.split("_", 1)[0].strip().upper()
    try:
        candles = pd.read_csv(p, encoding="utf-8-sig")
        if candles.empty or "time" not in candles.columns or "close" not in candles.columns:
            continue
        candles["time"] = pd.to_datetime(candles["time"], errors="coerce", utc=True)
        candles = candles.dropna(subset=["time"]).sort_values("time")
        if candles.empty:
            continue
        last = candles.iloc[-1]
        last_rows.append({
            "ticker": ticker,
            "last_close": float(last["close"]) if pd.notna(last["close"]) else None,
            "last_close_time": last["time"].date().isoformat(),
        })
    except Exception:
        continue

last_df = pd.DataFrame(last_rows)

df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()
if not last_df.empty:
    df = df.merge(last_df, how="left", on="ticker")

cols = [
    "ticker",
    "name",
    "last_close",
    "market_capitalization",
    "pe_ratio_ttm",
    "price_to_book_ttm",
    "roe",
    "beta",
    "free_float",
    "2024, Выручка, RUB",
    "2024, Чистая прибыль (убыток), RUB",
    "2024, Активы  всего, RUB",
]

cols = [c for c in cols if c in df.columns]
view = df[cols].copy()

rename = {
    "ticker": "Тикер",
    "name": "Компания",
    "last_close": "Цена акции (RUB)",
    "market_capitalization": "Капитализация (RUB)",
    "pe_ratio_ttm": "P/E (TTM)",
    "price_to_book_ttm": "P/B (TTM)",
    "roe": "ROE, %",
    "beta": "Бета",
    "free_float": "Free-float",
    "2024, Выручка, RUB": "Выручка 2024 (RUB)",
    "2024, Чистая прибыль (убыток), RUB": "Чистая прибыль 2024 (RUB)",
    "2024, Активы  всего, RUB": "Активы 2024 (RUB)",
}
view = view.rename(columns=rename)

if "Капитализация (RUB)" in view.columns:
    view = view.sort_values("Капитализация (RUB)", ascending=False)

fmt = {}
for c in ["Капитализация (RUB)", "Выручка 2024 (RUB)", "Чистая прибыль 2024 (RUB)", "Активы 2024 (RUB)"]:
    if c in view.columns:
        fmt[c] = "{:,.0f}"
if "Цена акции (RUB)" in view.columns:
    fmt["Цена акции (RUB)"] = "{:,.2f}"
for c in ["P/E (TTM)", "P/B (TTM)", "Бета", "Free-float", "ROE, %"]:
    if c in view.columns:
        fmt[c] = "{:.2f}"


view.head(91).style.format(fmt)


,Тикер,Компания,Цена акции (RUB),Капитализация (RUB),P/E (TTM),P/B (TTM),"ROE, %",Бета,Free-float,Выручка 2024 (RUB),Чистая прибыль 2024 (RUB),Активы 2024 (RUB)
28,SBER,Сбер Банк,304.75,"6,578,622,403,000",3.96,0.84,22.54,0.50,0.48,nan,nan,nan
21,ROSN,Роснефть,399.15,"4,274,245,113,596",6.29,0.46,7.43,0.65,0.11,"9,344,986,031,000","542,111,474,000","14,206,963,697,000"
44,PLZL,Полюс,"2,675.60","3,644,754,951,079",10.88,15.69,225.23,0.43,0.22,nan,"706,146,071,000","2,046,120,662,000"
85,LKOH,ЛУКОЙЛ,"5,184.50","3,623,341,502,379",6.61,0.65,8.66,0.65,0.55,"3,046,943,699,000","732,516,214,000","2,863,934,119,000"
62,NVTK,НОВАТЭК,"1,189.70","3,568,570,441,800",9.46,1.20,13.09,0.92,0.21,"938,160,995,000","316,457,142,000","2,524,167,022,000"
29,GAZP,Газпром,127.59,"3,022,634,127,072",2.09,0.16,7.95,1.00,0.47,"6,256,625,972,000","-1,076,329,869,000","26,162,827,395,000"
40,GMKN,Норильский никель,157.72,"2,494,730,639,040",15.01,2.71,19.94,0.73,0.33,"905,468,108,000","122,949,634,000","2,085,123,369,000"
33,SIBN,Газпром нефть,505.95,"2,408,580,216,612",0.00,0.76,16.01,0.41,0.05,"3,791,566,978,000","431,972,457,000","2,945,166,304,000"
10,TATN,Татнефть,563.40,"1,240,110,746,440",5.86,0.97,16.31,0.75,0.32,"1,563,778,385,000","251,375,287,000","1,493,117,675,000"
49,PHOR,ФосАгро,"6,422.00","831,131,000,000",7.21,3.79,58.45,0.34,0.26,"71,774,425,000","74,665,778,000","291,883,600,000"


### Описание ключевых переменных итогового датасета

Всего датасет содержит 156 переменных, ключевые из них:

**Идентификация компании**
- **ticker** — биржевой тикер компании, используется как основной идентификатор при объединении источников  
- **name** — наименование компании  

**Рыночные характеристики**
- **last_close** — последняя доступная цена закрытия акции  
- **market_capitalization** — рыночная капитализация компании  
- **average_daily_volume_last_4_weeks** — средний дневной объём торгов за последний месяц  
- **free_float** — доля акций в свободном обращении  
- **beta** — бета-коэффициент акции относительно рынка  

**Биржевые мультипликаторы (TTM / MRQ)**
- **pe_ratio_ttm (P/E)** — отношение цены к прибыли  
- **price_to_book_ttm (P/B)** — отношение цены к балансовой стоимости капитала  
- **price_to_sales_ttm (P/S)** — отношение цены к выручке  
- **roe** — рентабельность собственного капитала  
- **roa** — рентабельность активов  
- **net_margin_mrq** — чистая маржа  

**Финансовые результаты (СПАРК, 2024)**
- **2024, Выручка, RUB** — выручка компании  
- **2024, Чистая прибыль (убыток), RUB** — чистая прибыль  
- **2024, EBIT, RUB** — прибыль до уплаты процентов и налогов  
- **2024, Валовая прибыль (убыток), RUB** — валовая прибыль  

**Финансовое положение (СПАРК, 2024)**
- **2024, Активы всего, RUB** — совокупные активы компании  
- **2024, Капитал и резервы, RUB** — собственный капитал  
- **2024, Совокупный долг, RUB** — общий объём заёмных средств 